In [1]:
import numpy as np

class DropoutLayer:
    def __init__(self, p: float):
        """Initialize the dropout layer with dropout probability p."""
        self.p = p              # dropout probability
        self.mask = None        # mask will be generated during forward pass
        self.scale = 1 / (1 - p)

    def forward(self, x: np.ndarray, training: bool = True) -> np.ndarray:
        """Forward pass of the dropout layer."""
        if training:
            # generate mask: 1 means keep neuron, 0 means drop
            self.mask = (np.random.rand(*x.shape) > self.p).astype(x.dtype)
            return x * self.mask * self.scale
        else:
            # during inference dropout is disabled
            self.mask = None
            return x

    def backward(self, grad: np.ndarray) -> np.ndarray:
        """Backward pass — apply same dropout mask and scaling to gradients."""
        if self.mask is None:
            # If backprop happens during inference, gradients flow unchanged
            return grad
        return grad * self.mask * self.scale

#Usage example
np.random.seed(0)
drop = DropoutLayer(p=0.5)

x = np.array([1.0, 2.0, 3.0, 4.0])
grad = np.array([0.1, 0.2, 0.3, 0.4])

out = drop.forward(x, training=True)
grad_out = drop.backward(grad)

print("output =", out)
print("grad =", grad_out)

output = [2. 4. 6. 8.]
grad = [0.2 0.4 0.6 0.8]
